연세대학교 빅데이터 학회 YBIGTA <br>
작성자 : 이상헌 
# 언어 모델링에서의 N-gram 설명

## 1. 언어 모델링이란
"I love you." <br>

"I luv you."<br>

두 문자을 읽어보면, 결국 그 발음은 비슷함을 알 수 있다. <br>

하지만 단순히 위의 문장이 올바른 문장일 **확률**이 놈음을 우리는 경험적으로 할 수 있다. <br>

언어 모델링이란 말 그대로 언어(여기서 언어란 단어(어절)가 될 수도, 문장이 될 수도 있다) 의 확률을 구하는 것이다. <br>

이를 통하여 맞춤법 규칙, 띄어쓰기, 발음 변동 현상등을 해결할 수 있다. <br>

## 2. N-gram

**N gram**은 앞서 나온 단어들이 뒤의 단어들을 정하는데 영향을 미친다는 아이디어에서 나온 분석 방법이다. <br>

예를 들어, "피아노를 " 이라는 5글자(띄어쓰기 포함) 다음에는 "치"라는 글자가 나올 확률이 가장 높을 것이다. <br>

여기서 **N gram** 의 **N**은 총 앞선 몇개의 음절(혹은 어절도 가능하다)을 고려할 지를 정하는 단위이다. 이는 따라서 분야에 따라 매우 다른데, 보통 음성인식(음성)의 경우에는 3,4가 가장 좋은 방식이라는 것이 경험적으로 알려져 있다. <br>

몇 가지 예를 들어보자. <br>

"삼겹살에 소주가 먹고 싶어요." 를 2-gram(*bigram*이라고 부른다)로 분석한다면 <br>

> [삼,겹], [겹,살], [살,에], [에, ], [ ,소], ..., [어,요], [요,.] 가 될 것이다. <br>

이를 3-gram(trigram이 그 이름이다.)으로 분석한다면 <br>
> [삼,겹,살], [겹,살,에], [살,에, ], [에, ,소], ..., [어,요,.] 이 된다. <br>

엄청나게 긴 text를 해당 N-gram방식으로 분석하면, 같은 n-gram이 여러개 나오게 된다.<br>

그 n-gram들을 counting하고, 리스트 형태로 만든 것을 **frequency list**라고 부르게 된다. <br>
예) <br>
>[[나,는],100] <br>
>[[요,.],98] <br>
>... <br>
>[[니,혜],1] <br>

엄청나게 방대한 양의 글을 N-gram으로 트레이닝 시켰다는 가정을 하자. <br>

그 후에는 각 음절에 대한 확률을 구할 수 있게 된다. <br>

> $ \dfrac{해당 n gram의 빈도 수}{전체 n gram의 갯수}$ <rb>

예를 들면, 총 백만개의 3-gram model에서 love 라는 단어가 3번 나왔고, loving이라는 단어가 1번 나왔다고 하자.<br>

그렇다면 "l", "o", "v" 다음 e가 나올 확률은 $\dfrac{3}{4}$이 되는 것이다.


## 3. 언어모델로서의 N-gram

한 문장은 여러개의 음절로 이루어져 있다. 이 문장을 $W$라는 집합*(사실 여기서 집합이라는 표현은 맞지 않다. 집합은 원소의 순서가 없는데, N-gram에서는 음절간의 순서를 고려하기 때문이다. 따라서 리스트(python의 자료형)가 더 알맞은 표현이다)*이라고 표현하자. <br>

> $ W = \{w_1, w_2, w_3, ..., w_n \}$ <br>
> *여기서 $w_i$는 한 음절을 의미한다.*

그렇다면 우리는 해당 문장이 나오는 확률을 다음과 같이 구할 수 있다.

> $ P_{lm}(W) = \prod_i P(w_i)$
> *사실은 $ P_{lm}(W) = \prod_i P(w_i|\sigma(h_i))$ 라는 표현이 더 알맞은 표현이지만, 수학적으로 너무 깊게 들어가지 말자.*

이 문장의 확률이 의미하는 바를 말로 풀어보자면, **해당 문장이 옳은 문장인가, 를 판단할 수 있는 값**이라고 볼 수 있다. <br>

즉, "I luv you" 의 확률은 "I love you"의 확률보다 훨씬 떨어지게 되는 것이다. <br>

연세대학교 빅데이터 학회 YBIGTA <br>
작성자 : 이상헌 
# Beam search Algorithm이란

## 1. 음성인식 모델 및 언어 모델의 한계

Beam search를 논하기 이전, 모델의 한계를 짚고 넘어가자. <br>

결국 이 모델들은 문자열의 다음에 어떠한 글자가 올 지 확률상으로 예측하는 과정이다. <br>

물론 RNN의 경우에는 bidirectional, 즉 뒤에 나오는 말들 역시 고려가 되는 형태로 짜여져 있지만 <br>

감정적으로 명확한 이해를 위해 다음과 같은 예시를 들어보자. <br>



>"l", "o", "v" 다음 나올 글자의 경우를 예측하고자 한다. <br>
위의 예시를 따르면, <br>
$P("e") = 0.75$<br>
$P("i") = 0.25$<br>
$P(else) = 0 $<br>
$\therefore$ love가 모델에 따라 생성.

위의 과정에서는 loving이라는 글자를 예측할 방법을 해당 모델이 가지고 있지 않다. <br>

## 2. 해결책 및 복잡도

그렇다면 이 문제를 해결하기 위해서  "i"라는 후보를 버리지 않은 채 그 다음 글자를 예측하고자 하였다. <br>

>$P(lovea) = P(e|lov) * P(a|love)$<br>
...<br>
$P(loven) = P(e|lov) * P(n|love)$<br>
...<br>
$P(lovia) = P(i|lov) * P(a|lovi)$<br>
...<br>
$P(lovin) = P(i|lov) * P(n|lovi)$<br>
...<br>

후보를 버리지 않은 채 계산을 하게 된다면, 결국 다섯번 째 음가가 n임이 확실할 때 <br>
*후보군들을 hypothesis라고 부른다.*

$P(loven)=0 < P(lovin)$ <br>

이 되어서 "loving"이라는 글자를 계산할 수 있게된다. <br>

하지만 후보를 버리지 않는다는 것은 그 size가 커질 때마다 exponential로 계산량이 늘어난다. <br>

따라서 이를 해결하기 위한 방식이 beam search이다.

## 3. Beam search algorithm
![title](./beamsearch.JPG)

위의 그림을 감정적으로 이해해보자.<br>

A 다음 나올 글자를 예측하고자 하는데, 전체 후보 B, C, D 중 가장 확률이 높은 후보(hypothesis)는 C 이다. <br>

여기서 모든 후보를 버리지 않고, D 다음 나올 후보 E,F 중 가장 큰 후보가 F라고 하였을 때, 가장 확률이 높은 F만을 채택, <br> 

$P(D|A) = $P(F|D) * P(E|D)$ 가 된다. <br>

즉, E의 후보는 버릴 수 있으므로 복잡도가 exponential로 증가하지는 않는다. <br>

몇 번째 까지 후보군(hypothesis)를 고려한 것인가를 정하는 hyperparameter를 beam size라고 한다. <br>

위 그림으로 보자면 beam size = 2 가 되는 것이다. <br>

마찬가지로 다른 후보들도 진행을 하여, beam size만큼의 후보군들을 계산하면 복잡도는 $O(beamsize^2)$이 될 것이다. <br>

추가자료 : https://www.youtube.com/watch?v=UXW6Cs82UKo